In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
import logging
import seaborn as sns
import numpy as np
import cmdstanpy

from baynes.plotter import FitPlotter
from baynes.toyMC import SpectraSampler
from baynes.analysis import standard_analysis, multithreaded_run
from baynes.probability import HoSpectrum, hdi, ptolemy
from baynes.model_utils import get_model
from numba import njit

from sympy import symbols, sin, diff, erf, exp, sqrt, Piecewise, And
from sympy.utilities.lambdify import lambdify
from scipy.special import gamma
from scipy.constants import elementary_charge, hbar, c, electron_mass


import matplotlib.pyplot as plt

cmdstanpy.utils.get_logger().setLevel(logging.ERROR)
coeffs = np.loadtxt("parameters.dat")[:,1:]



In [ ]:
def allowed_beta(E, m_nu, Q=18589.6, Z=1):
    me = 510998.95
    p = np.sqrt(E**2+E*me)
    alpha = 1/137
    eta = Z* alpha * E/p
    F = 2*np.pi/(1-np.exp(-2*np.pi*eta))
    return F*p*(E+me)*(Q-E)*np.sqrt(np.clip((Q - E) ** 2 - m_nu**2, 0, None)) + 1e-8


def allowed_betasy():
    E = symbols("E")
    m_nu, Q = symbols("m_nu, Q")
    me = 510998.95
    p = sqrt(E**2+E*me)
    alpha = 1/137
    eta = alpha * E/p
    F = 2*np.pi/(1-exp(-2*np.pi*eta))
    return Piecewise((F*p*(E+me)*(Q-E)*sqrt((Q - E) ** 2 - m_nu**2), And(Q-E>=m_nu, m_nu>=0, Q>=0)),(0, True))+1e-8


m_nu = symbols("m_nu")
E = symbols("E")
Q = symbols("Q")
partial_m_beta = diff(allowed_betasy(), m_nu)
dens_beta = 1/allowed_betasy() * (partial_m_beta**2)
f_beta = lambdify((E, m_nu, Q), dens_beta, "numpy")

In [ ]:
def ptolemysym(coeffs):
    """Compute the b-decay spectrum of tritium in graphene."""
    E = symbols("E")
    y = symbols("y")
    m_nu, Q = symbols("m_nu, Q")
    me = 510998.95
    mhe3 = 2809413505.67592
    lambda_val = 4.21e-5
    eps0 = 5.76

    i_disc = lambda_val / (2 * np.pi**3)
    pb = sqrt(E**2 + 2 * E * me)

    y = 0
    for k in range(64):
        Qn = Q - coeffs[k][0]
        pn = coeffs[k][1]
        an = coeffs[k][2]
        bn = coeffs[k][3]
        cn = coeffs[k][4]
        E_nu = Qn - E
        xb = (pb - pn) / pn
        y += Piecewise((i_disc * pb * (E + me) * sqrt(E_nu**2 -m_nu**2) * E_nu * (an + xb * lambda_val * pn * (2 * bn - lambda_val * pn * cn)), E_nu>=m_nu), (0, True))

    i_cont = 1 / (np.pi**(7 / 2))
    QKE = Q - E - eps0
    b = -pb + sqrt(2 * mhe3 * (QKE - m_nu))
    bl = b * lambda_val
    pl = pb * lambda_val
    expbl = exp(-bl**2)
    exppl = exp(-pl**2)
    erfplbl = erf(pl) + erf(bl)
    pb2_2mhe3 = pb**2 / (2 * mhe3)
    kinf2 = sqrt(1 - (m_nu / (4.0 + QKE - pb2_2mhe3))**2)
    I0 = sqrt(np.pi) / (2 * lambda_val) * pb * (QKE - pb2_2mhe3)**2 * erfplbl
    I1 = 1 / (2 * lambda_val**2) * (QKE - pb2_2mhe3) * (QKE - 5.0 * pb2_2mhe3) * (exppl - expbl)
    I2 = 1 / (4 * lambda_val**3) * ((5.0 * pb2_2mhe3  - 3.0 * QKE)* pb / mhe3) * (
            -2 * pl * exppl - 2 * bl * expbl + sqrt(np.pi) * erfplbl)
    I3 = 1 / (2 * lambda_val**4) * ((5.0 * pb2_2mhe3 - QKE )/ mhe3) * (
            exppl * (1 + pl**2) - expbl * (1 + bl**2))
    I4 = 5 * pb / (32 * mhe3**2 * lambda_val**5) * (
            -expbl * (6 * bl + 4 * bl**3) - exppl * (6.0 * pl + 4 * pl**3) + 3 * sqrt(np.pi) * erfplbl)
    I5 = 1 / (8 * mhe3**2 * lambda_val**6) * (exppl * (2 + 2 * pl**2 + pl**4) - expbl * (2.0 + 2.0 * bl**2 + bl**4))
    y += Piecewise((i_cont * (E + me) * kinf2 * (I0 + I1 + I2 + I3 + I4 + I5), QKE>=m_nu), (0, True)) + 1e-8

    return y

m_nu = symbols("m_nu")
E = symbols("E")
Q = symbols("Q")

partial_m_pt = diff(ptolemysym(coeffs), m_nu)
dens_pt = 1/ptolemysym(coeffs) * (partial_m_pt**2)
f_pt = lambdify((E, m_nu, Q), dens_pt, "numpy")


In [ ]:
m_nu = 0.05
Q = 18589.6
E = np.linspace(18570, Q, 1000)

pt_norm = f_pt(18000,m_nu, Q)
beta_norm = f_beta(18000,m_nu, Q)

sp1 = ptolemy(E, coeffs, m_nu, Q)
beta = allowed_beta(E, 0)
beta_d = np.array([f_beta(e,m_nu, Q) for e in E])
pt_d = np.array([f_pt(e, m_nu, Q) for e in E])

#for m in [0, 0.05, 0.1, 0.5, 1, 2]:
 #   plt.plot(E, ptolemy(E, coeffs, m, Q))
#plt.plot(E, beta/sum(beta))
#plt.plot(E, sp1/sum(sp1))
plt.plot(E, beta_d)
plt.plot(E, pt_d*beta_norm/pt_norm)
plt.yscale("log")


In [ ]:
m = 0.0001
A = 3.57e8
A_T = A * 1
bkg = 0
FWHM= 0.1
n_days = 3* 365
Q=18592.01 
s_Q = 0.07


s = SpectraSampler({'$^{163}-Ho$': [allowed_beta, [0], A_T]}, flat_bkg=bkg, FWHM=FWHM, dE=0.02, integrate=True, ROI=[18580, 18591])
s.plot_spectrum()
s.set_measure_time(n_days, n_det=1)

events = s.sample()[0]
s.plot_events(events)

In [ ]:
m_nu=1
Q=18592.01 

E = np.arange(Q-5, Q, 0.02)
sp1 = ptolemy(E, coeffs, m_nu, Q)
plt.plot(E, sp1)
E = np.arange(Q-5, Q, 0.01)
sp1 = ptolemy(E, coeffs, m_nu, Q)
plt.plot(E, sp1)
plt.yscale("log")

In [ ]:
m = 0.05
A = 3.57e8
A_T = A * 100
bkg = 0
FWHM= 0.1
n_days = 3* 365
Q=18592.01 
s_Q = 0.07
coeffs = np.loadtxt("parameters.dat")[:,1:]


s = SpectraSampler({'$^{163}Ho$': [ptolemy, [coeffs, m, Q], A_T]}, flat_bkg=bkg, FWHM=FWHM, dE=0.05, integrate=False, ROI=[Q-6, Q+0.5])
s.plot_spectrum()
s.set_measure_time(n_days, n_det=1)

events = s.sample()[0]
s.plot_events(events)

data={'N_bins': len(events),
      'x': s.ROI_bin_centers,
      "N_ev": s.n_events,
      'counts': events,
      'p_Q': Q,
      'p_std_Q': s_Q,
      'p_FWHM': FWHM,
      'p_std_FWHM': 0.01,
      'm_max':10,
      "coeffs": coeffs,
      "prior":1
}

In [ ]:
model= get_model("ptolemy_endpoint_allQs.stan")



sampler_kwargs={
    'chains': 4,
    'iter_warmup': 500,
    'iter_sampling': 500,
    'save_warmup': True,
    'adapt_delta': 0.9,
    "sig_figs": 10,
    "show_console": False,
    "max_treedepth": 12,
    "inits": {"m_red": 0.01, "z": 0, "Qs":np.linspace(0.001, 6, 63)}
}

plot_pars = ['m_nu', 'Q', "Qs[1]", "Qs[2]", "Qs[62]", "Qs[63]"]
p = FitPlotter(col_wrap=4)

#fit = model.sample(data, chains=1, fixed_param=True , iter_sampling=4, iter_warmup=1)

fit = standard_analysis(model, data, p, sampler_kwargs, fit_title='m=0', plot_params = plot_pars)

In [ ]:
plot_pars = ['m_nu', 'Q', "Qs[1]", "Qs[2]", "Qs[62]", "Qs[63]"]
p.pair_grid(parameters=plot_pars)

In [ ]:
model= get_model("ptolemy_endpoint.stan")

sampler_kwargs={
    'chains': 4,
    'iter_warmup': 500,
    'iter_sampling': 500,
    'save_warmup': True,
    'adapt_delta': 0.9,
    "sig_figs": 10,
    "show_console": False,
    "inits": {"m_red": 0.01, "z": 0}
}

plot_pars = ['m_nu', 'Q']
p = FitPlotter(col_wrap=4)

#fit = model.sample(data, chains=1, fixed_param=True , iter_sampling=4, iter_warmup=1)

fit_noQs = standard_analysis(model, data, p, sampler_kwargs, fit_title='m=0', plot_params = plot_pars)

In [ ]:
p.pair_grid(parameters=['m_nu', 'Q'])

In [ ]:
m = 0.05
A = 3.57e8
A_T = A * 100
bkg = 0
FWHM= 0.1
n_days = 3* 365
Q=18592.01 
s_Q = 0.07
coeffs = np.loadtxt("parameters.dat")[:,1:]
model= get_model("ptolemy_endpoint.stan")


def nu_mass_fit(m):

    s = SpectraSampler({'$^{163}Ho$': [ptolemy, [coeffs, m, Q], A_T]}, flat_bkg=bkg, FWHM=FWHM, dE=0.01, integrate=False, ROI=[Q-5, Q+0.5])
    s.set_measure_time(n_days, n_det=1)
    events = s.sample()[0]

    data={'N_bins': len(events),
          'x': s.ROI_bin_centers,
          "N_ev": s.n_events,
          'counts': events,
          'p_Q': Q,
          'p_std_Q': s_Q,
          'p_FWHM': FWHM,
          'p_std_FWHM': 0.01,
          'm_max':10,
          "coeffs": coeffs,
          "prior":0
    }


    inits={}
    inits['m_nu_red'] = np.random.uniform(0,0.05)
    inits['z'] = np.random.normal(0, 0.1)

    sampler_kwargs={
        'chains': 1,
        'iter_warmup': 500,
        'iter_sampling': 1000,
        'save_warmup': False,
        'adapt_delta': 0.9,
        "sig_figs": 10,
        "show_console": False,
        "show_progress": False,
        "inits": inits
    }

    fit = model.sample(data, **sampler_kwargs)
    return fit

n_fits = 64
n_processes = 32
result = multithreaded_run(nu_mass_fit, [m]*n_fits, n_processes)

In [ ]:
a=1
def f():
    return a

In [ ]:
data = {}
efficiency = [0.1, 1, 10, 100]
for e in efficiency:
    A_T = A * e
    data[e] = multithreaded_run(nu_mass_fit, [m]*n_fits, n_processes)

data_array = {}
for key, item in data.items():
    posteriors = []
    for fit in item:
        posteriors.append(fit.draws_pd()['m_nu'].to_numpy().flatten())
    posteriors = np.array(posteriors)
    data_array[key] = posteriors
dict_to_json(data_array, "ptolemy/m0.json")

In [ ]:
data = {}
m_nus = [0.05, 0.1, 0.3, 0.5]
for m in m_nus:
    A_T = A * 100
    data[m] = multithreaded_run(nu_mass_fit, [m]*n_fits, n_processes)

m_data_array = {}
for key, item in data.items():
    posteriors = []
    for fit in item:
        posteriors.append(fit.draws_pd()['m_nu'].to_numpy().flatten())
    posteriors = np.array(posteriors)
    m_data_array[key] = posteriors
dict_to_json(m_data_array, "ptolemy/msweep100.json")

In [ ]:
from baynes import dict_to_json

data_array = {}
for key, item in data.items():
    posteriors = []
    for fit in item:
        posteriors.append(fit.draws_pd()['m_nu'].to_numpy().flatten())
    posteriors = np.array(posteriors)
    data_array[key] = posteriors
dict_to_json(data_array, "ptolemy/m0.json")

### Plot the posteriors for $m_\nu$ and combine their samples in a histogram. The upper limit on the parameter's estimate is given by the confidence interval of this distribution.

In [ ]:
posteriors = []
for fit in data[0.05]:
    posteriors.append(fit.draws_pd()['m_nu'].to_numpy().flatten())
posteriors = np.array(posteriors)

ax = p.new_figure('multi').subplots()
sns.boxplot(posteriors.transpose(), whis=[2.5, 95], showfliers=False, palette='flare', ax=ax)
ax.figure.set_size_inches(25, 5)
ax.set_xticks([])
ax.set_ylabel('m_nu', fontsize=25)


full = posteriors.flatten()
ax = p.new_figure('multi').subplots()
x_max=1

prob = 0.68
print( str(prob*100) + '% highest density interval: ', hdi(full, prob=prob))
ax.axvspan(hdi(full, prob=prob)[1], x_max, color='gray', alpha=0.2, lw=0)

prob = 0.9
print( str(prob*100) + '% highest density interval: ', hdi(full, prob=prob))
ax.axvspan(hdi(full, prob=prob)[1], x_max, color='gray', alpha=0.2, lw=0)

sns.histplot(posteriors.transpose(), bins=1000, alpha=1, multiple='stack', legend=False, lw=0., palette='flare', ax=ax)
ax.grid(False)
ax.set_xlim(0, x_max)
ax.set_xlabel('m_nu', fontsize=14)
ax.set_ylabel('counts', fontsize=14)
ax.figure.set_size_inches(8, 5)

In [ ]:
his = []
for key, posteriors in m_data_array.items():
    lo, hi = hdi(posteriors.flatten(), prob=0.9)
    his.append(hi)
    plt.scatter([key, key], [lo,hi])
#plt.xscale("log")

In [ ]:
effs = np.log10(np.array(efficiency))
pars = np.polyfit(np.log10(np.array(efficiency)), his, 1)

In [ ]:
plt.plot(effs, np.polyval(pars, effs))

In [ ]:
pars

In [ ]:
help(np.polyval)